In [2]:
from enhanced_hybrid_hhl.quantum_linear_system import QuantumLinearSystemProblem
from utils.QuantumLinearSystemSolver import QuantumLinearSystemSolver
from HHL import HHL
from get_result import get_fidelity_result, get_estimator_result
from inversion_circuits import HybridInversion

ModuleNotFoundError: No module named 'enhanced_hybrid_hhl'

In [4]:
A = [[0.5,0.1],[0.1,0.5]]
b = [1,0]

In [5]:
test_problem = QuantumLinearSystemProblem(A_matrix=A, b_vector=b)

In [6]:
E_Hybrid_HHL = HHL(get_result=get_fidelity_result,
                   pre_processing=ideal_preprocessing,
                   eigenvalue_inversion=HybridInversion)

In [7]:
hybrid_result = E_Hybrid_HHL.estimate(problem = test_problem,
                      num_clock_qubits=3,
                      )

In [8]:
print(hybrid_result)

{   'circuit_results': {'0': 431, '1': 593},
    'eigenbasis_projection_list': [0.7071067811865476, 0.7071067811865476],
    'eigenvalue_list': [0.4, 0.6],
    'ideal_x_statevector': None,
    'post_processing': None,
    'results_processed': (0.9973337235915666+0j),
    'rotation_angle_list': None,
    'shots': None}


In [9]:
cann_HHL = HHL(get_fidelity_result)
cann_result = cann_HHL.estimate(problem = test_problem,
                  num_clock_qubits=3,
                  max_eigenvalue=1)

In [14]:
print(abs(cann_result.results_processed))

0.9789194176139707


In [24]:
from qiskit_ibm_runtime import QiskitRuntimeService
 
service = QiskitRuntimeService()
backend = service.backend("ibmq_qasm_simulator")

#simulator = AerSimulator().from_backend(torino)

In [25]:
x_statevector = QuantumLinearSystemSolver(problem=test_problem).ideal_x_statevector
print(x_statevector.data)
print(x_statevector.to_operator().data)

[ 0.98058068+0.j -0.19611614+0.j]
[[ 0.96153846+0.j -0.19230769-0.j]
 [-0.19230769+0.j  0.03846154+0.j]]


In [26]:
from qiskit.quantum_info import Statevector

def statevector_to_observable(statevector, num_clock_qubits):

    zero_statevector = Statevector([1, 0])
    one_statevector = Statevector([0,1])

    final_statevector = Statevector([0,1])
    for _ in range(num_clock_qubits):
        final_statevector = final_statevector.expand(zero_statevector)

    final_statevector = final_statevector.expand(statevector)

    return final_statevector.to_operator() 

print(statevector_to_observable(x_statevector,3))

Operator([[0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
           0.        -0.j, 0.        -0.j, 0.        -0.j],
          [0.        +0.j, 0.96153846+0.j, 0.        +0.j, ...,
           0.        -0.j, 0.        -0.j, 0.        -0.j],
          [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
           0.        -0.j, 0.        -0.j, 0.        -0.j],
          ...,
          [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
           0.        +0.j, 0.        +0.j, 0.        +0.j],
          [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
           0.        +0.j, 0.        +0.j, 0.        +0.j],
          [0.        +0.j, 0.        +0.j, 0.        +0.j, ...,
           0.        +0.j, 0.        +0.j, 0.        +0.j]],
         input_dims=(2, 2, 2, 2, 2), output_dims=(2, 2, 2, 2, 2))


In [27]:
projection_operator = statevector_to_observable(x_statevector,3)

estimator_hhl = HHL(get_result=get_estimator_result(backend=backend, observable=projection_operator))

In [28]:
estimator_result = estimator_hhl.estimate(test_problem,
                       3,
                       1,
                       quantum_conditional_logic=False)

In [29]:
print(estimator_result)

{   'circuit_results': 0.5038461538461538,
    'eigenbasis_projection_list': None,
    'eigenvalue_list': None,
    'ideal_x_statevector': None,
    'post_processing': None,
    'results_processed': -0.212,
    'rotation_angle_list': None,
    'shots': None}
